In [ ]:
# importing requirements
!pip install wget
import wget
import torch
import gc
torch.manual_seed(0) #For reproducibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# download the dataset 
# will take around 7 min
url='https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/rscbjbr9sj-2.zip'

wget.download(url, '/content/Data.zip')

'/content/Data (1).zip'

In [ ]:
# unzipping the files
!unzip /content/Data.zip -d /content/
!gzip -d /content/OCT2017.tar.gz
!tar -xf /content/OCT2017.tar -C /content

Archive:  /content/Data.zip
  inflating: /content/OCT2017.tar.gz  
replace /content/ChestXRay2017.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/ChestXRay2017.zip  
replace /content/code2017.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/code2017.zip   


In [ ]:
from MyDataSet import OCTDataset

train_set = OCTDataset('/content/OCT2017/train')

for x, y in train_set:
  print(x.shape)
  break
gc.collect()
# CNV 37205
# DME 11348
# DRUSEN 8616
# NORMAL 26315

valid_set = OCTDataset('/content/OCT2017/test')

for x, y in valid_set:
  print(x.shape)
  break

# CNV 250
# DME 250
# DRUSEN 250
# NORMAL 250
gc.collect()

In [ ]:
batch_size = 64
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
from MyResNet import ResidualBlock, ResNet, BottleneckBlock, ResNet50

# For ResNet18:
model=ResNet(ResidualBlock,[2,2,2,2],4).to(device)
# For ResNet34:
# model=ResNet(ResidualBlock,[3,4,6,3],4).to(device)
# For ResNet50:
# model=ResNet50(BottleneckBlock,[3,4,6,3],4).to(device)

In [ ]:
num_iter = 10
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_iter, eta_min=5e-6)
scaler = torch.cuda.amp.GradScaler()

In [ ]:
from TrainnTest import train, test
best_accuracy = 0.0
train_acc_list, test_acc_list = [], []
train_loss_list, test_loss_list = [], []

gc.collect()
for epoch in range(num_iter):
    print("\nEpoch: {}/{}".format(epoch+1, num_iter))

    train_loss, train_acc = train(trainloader,model, loss_fn, optimizer, scaler,device)

    test_loss, test_acc = test(testloader,model, loss_fn,device)

    print("\nEpoch {}/{}: \nTrain Loss {:.04f}\t Train Accuracy {:.04f}\nValid Loss {:.04f}\t Valid Accuracy {:.04f}\nLearning Rate {:.08f}".format(
        epoch + 1,
        num_iter,
        train_loss,
        train_acc,
        test_loss,
        test_acc,
        float(optimizer.param_groups[0]['lr']))
    )
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    
    scheduler.step()

    if test_acc > best_accuracy:
      best_accuracy = test_acc
      torch.save({'model_state_dict':model.state_dict(),
                  'test_acc': test_acc, 
                  'epoch': epoch}, './mresnet_best.pth')

In [ ]:
# restore the labels from the numbers
dic = {
    0 : "CNV",
    1 : "DME" ,
    2 : "DRUSEN",
    3 : "NORMAL" 
}

In [ ]:
from ResultAnalysis import PlotEpoch, ShowMistakes,ShowConfusionMatrix,GetPredictions
PlotEpoch(test_acc_list,test_loss_list,train_acc_list,train_loss_list)

preds, labels = GetPredictions(model,testloader,device)
labels = [dic[i] for i in labels]

ShowConfusionMatrix(preds,labels)

ShowMistakes(preds,labels,valid_set)